In [29]:
%matplotlib inline

import pandas
import seaborn

seaborn.set()

In [79]:
!pwd

/home/jng/devel/peakrisk/posts


In [31]:
infile = '../files/light.csv'

In [80]:
!scp 192.168.0.133:Adafruit_Python_BMP/light.csv .
!mv light.csv ../files

light.csv                                     100%  381KB 380.6KB/s   00:00    


light.csv                                       0%    0     0.0KB/s   --:-- ETA

In [81]:
data = pandas.read_csv(infile, index_col='date', parse_dates=['date'])

In [82]:
data.describe()

              temp       pressure     altitude  sealevel_pressure
count  5694.000000    5694.000000  5694.000000        5694.000000
mean     28.695188  101575.399895   -25.514355      101723.042677
std       2.500549    2787.022287   171.204387        1643.212884
min      27.200000   28421.000000 -2067.737364       49539.000000
25%      28.000000  101637.000000   -34.410542      101638.000000
50%      28.500000  101695.000000   -30.841782      101697.000000
75%      29.300000  101739.000000   -25.943123      101740.000000
max     123.400000  128808.000000  7610.794710      139326.000000

In [83]:
#data.plot()
data.temp.plot()

In [84]:
data.altitude.plot()

In [85]:
data.temp[data.temp < 50.0].plot()

In [86]:
data.temp[data.temp > 50.0]

date
2015-06-29 00:30:04.757425    123.4
2015-06-29 02:16:16.421378     58.1
2015-06-29 04:36:31.999731     58.5
2015-06-29 06:55:47.389789     98.3
2015-06-29 07:27:50.925301     67.9
2015-06-29 09:35:04.905765     71.8
2015-06-29 11:27:17.365401     87.6
2015-06-29 13:17:29.549340     94.8
2015-06-29 13:33:31.345615     91.9
2015-06-29 13:49:33.137824     67.4
Name: temp, dtype: float64

In [87]:
!pwd

/home/jng/devel/peakrisk/posts


In [88]:
data.plot(subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fcba96b0240>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7fcba9779c88>], dtype=object)

In [95]:
mask = data.temp < 50.0

mask = mask & (abs(data.altitude) < 100.0)

good = data[mask]

In [96]:
good.plot(subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fcba8dab6a0>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7fcba8d83630>], dtype=object)

In [97]:
good.describe()

              temp       pressure     altitude  sealevel_pressure
count  5671.000000    5671.000000  5671.000000        5671.000000
mean     28.602557  101667.590196   -30.760775      101678.451067
std       0.756779    1573.798722     6.305853        1126.585671
min      27.200000   37993.000000   -56.873967       49539.000000
25%      28.000000  101637.000000   -34.410542      101638.000000
50%      28.500000  101695.000000   -30.841782      101696.000000
75%      29.300000  101739.000000   -25.943123      101739.000000
max      30.100000  128808.000000    31.768412      121592.000000

In [54]:
data.describe()

              temp       pressure     altitude  sealevel_pressure
count  5006.000000    5006.000000  5006.000000        5006.000000
mean     28.709149  101625.235118   -28.220942      101711.294846
std       1.592905    1894.670658    67.558175         870.452436
min      27.200000   37993.000000   -56.873967      101522.000000
25%      28.100000  101630.000000   -33.829665      101631.000000
50%      28.700000  101689.000000   -30.343719      101690.000000
75%      29.400000  101732.000000   -25.361774      101733.000000
max     123.400000  118018.000000  2728.365527      133319.000000

In [55]:
data.count()

temp                 5006
pressure             5006
altitude             5006
sealevel_pressure    5006
dtype: int64

In [57]:
mask.value_counts()

True     5001
False       5
dtype: int64

In [60]:
data.temp[mask == False]

date
2015-06-29 00:15:03.099947     28.7
2015-06-29 00:30:04.757425    123.4
2015-06-29 01:00:07.986653     28.6
2015-06-29 02:16:16.421378     58.1
2015-06-29 02:47:19.895202     28.6
Name: temp, dtype: float64

In [61]:
good.temp[mask == False]

Series([], Name: temp, dtype: float64)

In [64]:
type(good)

pandas.core.frame.DataFrame

In [65]:
data[data.temp > 50.0]

                             temp  pressure   altitude  sealevel_pressure
date                                                                     
2015-06-29 00:30:04.757425  123.4    101808 -40.134605             101811
2015-06-29 02:16:16.421378   58.1    101850 -43.368556             101838

In [66]:
data[data.altitude > 0.0]

                            temp  pressure     altitude  sealevel_pressure
date                                                                      
2015-06-29 00:15:03.099947  28.7     49788  2728.365527             133102
2015-06-29 01:00:07.986653  28.6     49695  2714.956421             133319
2015-06-29 02:47:19.895202  28.6     49721  2718.988121             133202

In [70]:
data[data.pressure > 110000.0]

                            temp  pressure   altitude  sealevel_pressure
date                                                                    
2015-06-29 00:00:01.421409  28.8    118018 -39.056399             101791
2015-06-29 02:31:18.059312  28.6    114827 -42.373601             101837

In [ ]:
data[data.pressure < 110000.0]